In [7]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    logging,
    TextDataset,
    DataCollatorForLanguageModeling
)


# from trl import SFTTrainer
import gc
import json
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.cuda.empty_cache()
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cuda")

from config import *
import itertools

eval_dataset_json_file = "/home/admin/blessed/questions_new_final.json"
output_dir = "/home/admin/blessed/home/ubuntu/checkpoint-1400"

In [5]:




# model = AutoModelForCausalLM.from_pretrained("checkpoint-2700", torch_dtype="auto", trust_remote_code=True).to(device)
tokenizer = AutoTokenizer.from_pretrained(output_dir, trust_remote_code=True)
# tokenizer.pad_token=tokenizer.eos_token
# tokenizer.padding_side="left"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    output_dir,
    quantization_config=bnb_config,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
    device_map={"": 0},
    # revision="refs/pr/23" #the main version of Phi-2 doesn’t support gradient checkpointing (while training this model)
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.42s/it]


In [9]:
with open(eval_dataset_json_file, "r") as f:
    test_data = f.read()

orig_test_data=json.loads(test_data)

In [10]:
all_options  = []
def fetch_question(example):
    global all_options
    data_ = {"questions": []}


    for ex in example:

        options = "\n"
        opts = []
        for key in example[ex].keys():
            if key.startswith("opt"):
                if example[ex][key] == None:
                    continue
                options += str(key) + " " +example[ex][key]+"\n"
                opts.append((example[ex][key], key.split("option ")[1]))

        all_options.append(opts)


        q = "Instruct:"+"\n"+example[ex]['question'] + "\n\n\nAbbreviations: \n"   +\
            '\n'.join(str(e) for e in  list(dict.fromkeys(example[ex]["abbreviation"])) )\
            +"\n\nConsidering the following retrieved contexts"+"\ncontext 1: "+example[ex]["context_qwen2"][0]+example[ex]["context_qwen2"][1]+"\ncontext 2: "+'\n'.join(example[ex]["context_gle"] ) + "\ncontext 3: "+example[ex]["context_bm"][0] +"\n"+ example[ex]['question'] + "\n" + options


        q += "\nOutput:" 

        #   ans = example[ex]["answer"] + "\n ### Answer: " + example[ex]["explanation"]

        data_["questions"].append(q)
        
    return data_
eval_dataset  = fetch_question(orig_test_data)


all_test = []
all_test_ids = []
prompt ="Instruct: Answer the following question using the context provided.Your answer must start with the correct option letter (A, B, C, D, or E) followed by the text of the answer."


prompt ="Instruct: Answer the following question using the context provided.Your answer must start with the correct option letter (A, B, C, D, or E) followed by the text of the answer."
for key in (orig_test_data):

  all_test_ids.append(int(key.split(' ')[1]))

In [ ]:


import random

import time
import pandas as pd
import torch
import random
from statistics import mode
submission = {"Question_ID": [], "Answer_ID": []}
batch_size = 1 # Adjust batch size as needed

options_ids = ["option 1", "option 2", "option 3", "option 4", "option 5"]
if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
# Set the model to evaluation mode once
model.eval()
# Group data into batches
bad_ids = []
for i in range(0, len(eval_dataset['questions']), batch_size):

    batch_question_ids = all_test_ids[i:i + batch_size]

    batch_prompts = eval_dataset['questions'][i:i + batch_size]
    string_opts = ' '.join(opt[0] for opt in all_options[i])
    # print(string_opts)
    if not ("option" in string_opts or "above" in string_opts):
        print("batch shuffle")
        # Generate all permutations of the list
        votes = []
        batch_prompts = []
        all_permutations = list(itertools.permutations(all_options[i]))
        all_permutations = random.sample(all_permutations, 20 if len(all_permutations)>20 else len(all_permutations))
        for option_set in all_permutations:
                option_map = []
                options = "\n"
               
                for z in range(len(option_set)):
                    options += options_ids[z] + " " +  option_set[z][0] + "\n"
                    option_map.append(option_set[z][1])

                batch_prompts.append(question_former("question "+str(all_test_ids[i]), options))

        model_inputs = tokenizer(batch_prompts, return_tensors="pt", padding=False,padding_side ="left" truncation=True).to("cuda")
        with torch.no_grad():
            results = model.generate(**model_inputs, pad_token_id=tokenizer.pad_token_id, max_new_tokens=5,)
        
        for file 
            batch_results = [tokenizer.decode(result, skip_special_tokens=True) for result in results]
            answer_id = int(batch_results[0].split("Output:")[1].split("option")[1][1])
            votes.append(option_map[answer_id-1])
            
            
            
            answer_id = int(mode(votes))
        submission["Answer_ID"].append(answer_id)
        submission["Question_ID"].append(batch_question_ids[0])
        
    else:

        batch_question_ids = all_test_ids[i:i + batch_size]
        # Tokenize the batch prompts
        model_inputs = tokenizer(batch_prompts, return_tensors="pt", padding=False, truncation=True).to("cuda")

        start = time.time()

        # Perform batch inference
        with torch.no_grad():
            results = model.generate(**model_inputs, pad_token_id=tokenizer.pad_token_id, max_new_tokens=5,)

        # Decode the batch results
        batch_results = [tokenizer.decode(result, skip_special_tokens=True) for result in results]
    
        # print(time.time() - start)
        # print(batch_results[0])
        
        # Extract the answer IDs from batch results
        for question_id, result_text in zip(batch_question_ids, batch_results):

            submission["Question_ID"].append(question_id)
            try:
                answer_id = int(result_text.lower().split("Output:")[1].split("option")[1][1])
                submission["Answer_ID"].append(answer_id)
            except:
                try:
                
                    answer_id = (result_text.lower().split("output:")[2].split("option")[1][1])

                    submission["Answer_ID"].append(int(answer_id))
                except:
                    print(result_text)
                    submission["Answer_ID"].append(random.randint(1, 4))
               
    
    print(i, answer_id)
    
# # # Create a DataFrame and save it to a CSV file
df_submission = pd.DataFrame(submission)
df_submission.to_csv("submission_test.csv", index=False)


In [ ]:
print(len(submission["Answer_ID"]))

In [ ]:
print(batch_results[0])

In [ ]:
list(range(len(submission["Question_ID"])))[-1]

In [ ]:
df_submission = pd.DataFrame(submission)
df_submission.to_csv("submission_test_halfway.csv", index=False)

In [ ]:
submission["Answer_ID"].append(2)
submission["Question_ID"].append(batch_question_ids[0])

In [ ]:
batch_question_ids[0]

In [ ]:
len(submission["Answer_ID"]

In [ ]:
print(batch_results[0].split("Output:")[1])

In [ ]:
print(mode(votes))

In [ ]:
len(submission["Answer_ID"])

In [ ]:
20 if len(all_permutations)>20 else len(all_permutations)


In [ ]:
model_inputs.keys()

In [ ]:
model_inputs

In [ ]:
len(submission["Answer_ID"])

In [ ]:
df_submission = pd.DataFrame(submission)
df_submission.to_csv("submission_full_batch_trick.csv", index=False)

In [ ]:
sum(submission["Answer_ID"][:100] == answers_df["Answer_ID"][:100])/len(submission["Answer_ID"])

In [ ]:
len(submission["Answer_ID"])

In [ ]:
new_best = pd.read_csv("new_best.csv")

In [ ]:
sum(new_best["Answer_ID"][:100] == answers_df["Answer_ID"][:100])/len(submission["Answer_ID"])


In [ ]:
from statistics import mode
 
def most_common(List):
    return(mode(List))

type(mode(votes))

In [ ]:
option_map

In [ ]:
print(option_set)

In [ ]:
option_set[0][1]

In [ ]:
all_permutations[10]

In [ ]:

# def fetch_question(example):
#     data_ = {"questions": []}

#     for ex in example:
#     #   print(len( example[ex]['question'].split("Please provide a detailed answer to the following question by starting with mentioning the correct option:")))
#         question  = example[ex]["question"]
#         # print(question, example[ex].keys())
#         options = "\n"
#         for key in example[ex].keys():
#            if key.startswith("opt"):
#               if example[ex][key] == None:
#                  continue
#               options += str(key) + " " +example[ex][key]+"\n"

#         q = "Instruct:Answer the following Telecommunication standards question considering the context provided, if the answer is not in the context, use your own Telecommunication/Networking knowledge to answer. :\n "+ question+ "\nAbbreviations: \n" +\
#             '\n'.join(str(e) for e in  list(dict.fromkeys(example[ex]["abbreviation"])) )\
#             +"\n\nConsidering the following retrieved contexts"+"\ncontext 1: "+ "\n".join(example[ex]["context_bm_300"])+ "\ncontext 2:"+'\n: '.join(example[ex]["context_gle"])+"\ncontext 3:"+'\n'.join(example[ex]["context_gte"][:4]) +"\n" + question + options
#             # +"\n\nConsidering the following retrieved contexts"+"\ncontext 1: "+example[ex]["context_gte"][0]+"\ncontext 2: "+'\n'.join(example[ex]["context_gle"]) + "\ncontext 3: "+example[ex]["context_bm"][0] +"\n"+ question + options
        


#         q += "\nOutput:" 

#         #   ans = example[ex]["answer"] + "\n ### Answer: " + example[ex]["explanation"]

#         data_["questions"].append(q)
        
#     return data_
# eval_dataset  = fetch_question(orig_test_data)
# # eval_dataset = Dataset.from_dict({
# #     'text': eval_dataset['questions'],
# # })


# all_test = []
# all_test_ids = []

# for key in (orig_test_data):
#   all_test_ids.append(int(key.split(' ')[1]))


In [ ]:
print(eval_dataset["questions"][0])

In [ ]:
import time
import pandas as pd
import torch
import random
submission = {"Question_ID": [], "Answer_ID": []}
batch_size = 1 # Adjust batch size as needed


if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
# Set the model to evaluation mode once
model.eval()

# Group data into batches
bad_ids = []
for i in range( 0, len(eval_dataset['questions']), batch_size):
    # batch_prompts = questions[i:i + batch_size]
    # batch_prompts = [raw_json['train'][key][0] + "\n ### Answer: " for key in batch_questions]
    # batch_question_ids = [int(key.split(" ")[1]) for key in batch_questions]

    batch_prompts = eval_dataset['questions'][i:i + batch_size]
    batch_question_ids = all_test_ids[i:i + batch_size]
    # Tokenize the batch prompts
    model_inputs = tokenizer(batch_prompts, return_tensors="pt", padding=False, truncation=True).to("cuda")

    start = time.time()

    # Perform batch inference
    with torch.no_grad():
        results = model.generate(**model_inputs, pad_token_id=tokenizer.pad_token_id, max_new_tokens=5,)

    # Decode the batch results
    batch_results = [tokenizer.decode(result, skip_special_tokens=True) for result in results]
   
    # print(time.time() - start)
    # print(batch_results[0])
    
    # Extract the answer IDs from batch results
    for question_id, result_text in zip(batch_question_ids, batch_results):
        # try:
      
        # except:
        #     print(result_text.split("Output:")[1])
        #     answer_id = random.randint(1,5)
        submission["Question_ID"].append(question_id)
        try:
            answer_id = int(result_text.split("Output:")[1].split("option")[1][1])
            submission["Answer_ID"].append(answer_id)
        except:
            try:
            
                answer_id = (result_text.lower().split("output:")[2].split("option")[1][1])

                submission["Answer_ID"].append(int(answer_id))
            except:
                 print(result_text)
                 submission["Answer_ID"].append(random.randint(1, 4))
                 bad_ids.append(i)
    if i == 365:
         break
    
    print(i, answer_id)
    
# # Create a DataFrame and save it to a CSV file
df_submission = pd.DataFrame(submission)
df_submission.to_csv("submission_test.csv", index=False)


In [ ]:
import pandas as pd
# Load the CSV files into DataFrames
answers_df = pd.read_csv('answers.csv')
submission_df = pd.read_csv('submission_test.csv')
print(len(answers_df), len(submission_df))
# Merge the DataFrames on the 'Question_ID' column
merged_df = pd.merge(answers_df, submission_df, on='Question_ID', suffixes=('_correct', '_submitted'))

# Calculate the number of correct answers
correct_answers = (merged_df['Answer_ID_correct'] == merged_df['Answer_ID_submitted']).sum()

# Calculate the total number of questions
total_questions = len(merged_df)

# Calculate accuracy
accuracy = correct_answers / total_questions

print(f'Accuracy: {accuracy:.2%}')

In [ ]:
submission_df

In [ ]:
all_test_ids[365]

In [ ]:
merged_df[merged_df['Answer_ID_correct'] != merged_df['Answer_ID_submitted']]

In [ ]:
submission["Answer_ID"]

In [ ]:
len(submission["Answer_ID"])
